Import the neccesary libraries

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2023-10-01'
end = '2024-09-30'

# Tickers of assets
assets = asset_classes = pd.read_csv("asset_classes10X41.csv").sort_values(by = ["Assets"])

assets = list(asset_classes["Assets"])

market_index = ['^J200.JO']

all_assets = assets + market_index
all_assets.sort()

# Downloading data
data = yf.download(all_assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = all_assets

[*********************100%***********************]  42 of 42 completed


In [3]:
# Calculating returns

Y = data[all_assets].pct_change().dropna()

display(Y.head())

,ABG.JO,ADH.JO,AMS.JO,APN.JO,ARI.JO,AVI.JO,BAW.JO,CLS.JO,DRD.JO,DSY.JO,...,SAP.JO,SOL.JO,SPP.JO,SSW.JO,TBS.JO,TFG.JO,TRU.JO,VOD.JO,WHL.JO,^J200.JO
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03 00:00:00+00:00,0.3912%,0.0476%,-4.0191%,-0.0059%,-1.8786%,-0.4306%,-2.8588%,0.5036%,-1.1206%,0.4076%,...,-2.8828%,-2.4006%,0.2939%,-6.8746%,-0.7448%,-1.4602%,-1.8592%,0.0866%,-0.3315%,-0.5611%
2023-10-04 00:00:00+00:00,-0.9479%,1.0000%,-0.2490%,0.7611%,0.0994%,-0.2838%,0.5208%,-0.1748%,-0.0667%,-0.5462%,...,6.0584%,-2.0968%,1.2343%,-1.4079%,-5.4788%,0.1357%,0.6315%,-0.2789%,-0.3024%,-0.2584%
2023-10-05 00:00:00+00:00,1.5794%,0.1886%,-2.1855%,-0.1171%,-0.7743%,0.8944%,0.6506%,0.8717%,1.8012%,0.5492%,...,0.2523%,-1.2026%,-0.1228%,-1.4280%,4.1383%,0.0834%,-2.9640%,1.6393%,0.6976%,1.0232%
2023-10-06 00:00:00+00:00,0.1734%,0.3294%,1.2089%,0.8265%,3.3013%,-1.2089%,1.5801%,-1.1458%,-0.1966%,0.3174%,...,-0.0687%,-0.7504%,3.6360%,-0.3524%,-0.4925%,-1.6035%,-0.2201%,-0.8444%,-0.8735%,0.5801%
2023-10-09 00:00:00+00:00,0.1616%,-1.2664%,5.6365%,-1.1395%,0.3551%,-0.4623%,1.9208%,-0.9873%,6.1720%,-1.1183%,...,-0.6412%,6.3508%,-2.3136%,7.6621%,1.0238%,-2.3386%,-1.5858%,-3.3585%,-1.4585%,0.0621%


In [4]:
import riskfolio as rp

# Building the portfolio object

port = rp.Portfolio(returns=Y[assets])
# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Create Turnover Constraints

port.allowTO = True # Allows to use Turnover Constraints
port.turnover = 0.05 # Maximum deviation in absolute value respect to benchmark weights
# By default benchweights is the equally weighted portfolio,
# if you want to use a different benchmark weights, you must
# specify a weights dataframe with assets names in columns
#port.benchweights = weights # Use a dataframe

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

,ABG.JO,ADH.JO,AMS.JO,APN.JO,ARI.JO,AVI.JO,BAW.JO,CLS.JO,DRD.JO,DSY.JO,...,RNI.JO,SAP.JO,SOL.JO,SPP.JO,SSW.JO,TBS.JO,TFG.JO,TRU.JO,VOD.JO,WHL.JO
weights,0.0000%,7.4390%,0.0000%,0.0000%,0.0000%,7.4390%,0.0000%,7.4390%,4.7170%,0.0000%,...,0.8086%,0.0000%,0.0000%,0.0000%,0.0000%,7.4390%,0.0000%,0.0000%,0.0000%,0.0000%
